# Customer.IO Webhook Processing

This notebook demonstrates how to process Customer.IO webhooks for delivery tracking and event handling.

## Setup

First, let's import the webhook processing utilities.

In [ ]:
import os
import sys
import json
import hmac
import hashlib
from datetime import datetime

# For local development with src imports (backwards compatibility)
# Add src directory to path if it exists
notebook_dir = os.path.dirname(os.path.abspath(''))
project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))
src_path = os.path.join(project_root, 'src')

if os.path.exists(src_path):
    sys.path.insert(0, src_path)
    # Try importing from src (local development pattern)
    try:
        from src.webhooks.processor import (
            verify_signature,
            parse_event,
            WebhookEvent
        )
    except ImportError:
        # Fall back to package imports
        from webhooks.processor import (
            verify_signature,
            parse_event,
            WebhookEvent
        )
else:
    # Databricks or package installation - use package imports
    from webhooks.processor import (
        verify_signature,
        parse_event,
        WebhookEvent
    )

print("Webhook processing utilities imported successfully")

## Databricks Environment Setup

This section handles the installation of required libraries and configuration for Databricks clusters. Skip this section if running locally.

In [ ]:
# Databricks library installation
# Uncomment the following lines when running on Databricks

# %pip install customerio-api-client
# OR if using a wheel file:
# %pip install /dbfs/path/to/customerio_api_client-1.0.0-py3-none-any.whl

# For development without packaging, install individual requirements:
# %pip install httpx>=0.25.0 pydantic>=2.0.0 pandas>=2.0.0 python-dotenv>=1.0.0 structlog>=24.0.0

# Restart kernel after installation if needed
# import dbutils
# dbutils.library.restartPython()

## Webhook Signature Verification

Verify that webhooks are authentically from Customer.IO.

In [ ]:
# Example webhook payload and signature
webhook_payload = '{"event_type": "email_delivered", "customer_id": "cust_123"}'
webhook_signature = "sha256=example_signature_here"
webhook_secret = "your_webhook_secret_here"

# Verify the signature
is_valid = verify_signature(
    payload=webhook_payload,
    signature=webhook_signature,
    secret=webhook_secret
)

print(f"Webhook signature valid: {is_valid}")

## Event Parsing

Parse webhook events and extract event types.

In [ ]:
# Parse the webhook event
event_data = parse_event(webhook_payload)
print(f"Parsed event data: {event_data}")

# Get the event type (import the function first)
from src.webhooks.processor import get_event_type
event_type = get_event_type(event_data)
print(f"Event type: {event_type}")

## Complete Webhook Processing Workflow

A complete example of processing an incoming webhook.

In [ ]:
def process_webhook(payload, signature, secret):
    """Process a Customer.IO webhook with full validation."""
    
    # Step 1: Verify signature
    if not verify_signature(payload, signature, secret):
        print("ERROR: Invalid webhook signature")
        return None
    
    print("SUCCESS: Webhook signature verified")
    
    # Step 2: Parse event
    try:
        event = parse_event(payload)
        event_type = get_event_type(event)
        
        print(f"Processing {event_type} event")
        
        # Step 3: Handle different event types
        if event_type == "email_delivered":
            print(f"Email delivered to customer {event.get('customer_id')}")
        elif event_type == "email_opened":
            print(f"Email opened by customer {event.get('customer_id')}")
        elif event_type == "email_clicked":
            print(f"Email link clicked by customer {event.get('customer_id')}")
        else:
            print(f"Received {event_type} event")
        
        return event
        
    except Exception as e:
        print(f"ERROR: Error processing webhook: {e}")
        return None

# Example usage
result = process_webhook(webhook_payload, webhook_signature, webhook_secret)
print(f"Processing result: {result}")